<a href="https://colab.research.google.com/github/BlackCurrantDS/Data-Mining/blob/master/Blood_Cell_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import PIL
import PIL.Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory

Loading dataset

In [2]:
train_ds = image_dataset_from_directory(
    directory='/content/drive/MyDrive/Blood Cell classification/dataset2-master/dataset2-master/images/TRAIN/',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256)
    )
validation_ds = image_dataset_from_directory(
    directory='/content/drive/MyDrive/Blood Cell classification/dataset2-master/dataset2-master/images/TEST_SIMPLE/',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256))
test_ds = image_dataset_from_directory(
    directory='/content/drive/MyDrive/Blood Cell classification/dataset2-master/dataset2-master/images/TEST/',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256))

Found 9977 files belonging to 4 classes.
Found 71 files belonging to 4 classes.
Found 2487 files belonging to 4 classes.
